# Mecanismo de Investimento Automático: Modelo de Investimento Magic Formula

* O que é um modelo de investimento?
* O que é um backtest?
* O que é Factor Investing?
    * Análise Fundamentalista Sistematizada
    * Blackrock, AQR...
* O que é o Fator Valor? E Fator Rentabilidade?
* EV/EBIT e ROIC

------

### Passo a passo:

   **Passo 1** - Importar os módulos e bibliotecas.
   
   **Passo 2** - Baixar os dados disponibilizados.
   
   **Passo 3** - Pegar as datas que iremos balancear a carteira. 
   
   **Passo 4** - Filtrar  a liquidez.
   
   **Passo 5** - Criar o ranking dos indicadores.
   
   **Passo 6** - Criar as carteiras. 
   
   **Passo 7** - Calcular a rentabilidade por carteira.
   
   **Passo 8** - Calcular a rentabilidade do modelo.

   **Passo 9** - Calcular a rentabilidade do ibovespa no mesmo período.

   **Passo 10** - Observar resultados de retorno.

   **Passo 11** - Otimizar nosso modelo com outros indicadores. 

   **Passo 12** - Observar métricas de risco, janelas móveis de retorno, eventos de estresse...

   ****
   
   
-------------------

# Passo 1: Importando os módulos necessários

In [ ]:
!pip install quantstats

In [ ]:
import pandas as pd
import quantstats as qs

# Passo 2: Baixar os dados dispobinilizados.

In [ ]:
dados_empresas = pd.read_csv("dados_empresas.csv")

dados_empresas

# Passo 3: Calcular os retornos mensais das empresas.

In [ ]:
dados_empresas["retorno_mensal"] = dados_empresas.groupby("ticker")["preco_fechamento_ajustado"].pct_change()
dados_empresas["retorno_mensal"] = dados_empresas.groupby("ticker")["retorno_mensal"].shift(-1)

dados_empresas

# Passo 4: Filtrar liquidez.

In [ ]:
dados_empresas = dados_empresas[dados_empresas["volume_negociado"] > 1000000]

dados_empresas

# Passo 5: Criar o ranking dos indicadores.

In [ ]:
dados_empresas["ranking_ebit_ev"] = dados_empresas.groupby("data")["ebit_ev"].rank(ascending = False)
dados_empresas["ranking_roic"] = dados_empresas.groupby("data")["roic"].rank(ascending = False)

dados_empresas

In [ ]:
dados_empresas["ranking_final"] = dados_empresas["ranking_ebit_ev"] + dados_empresas["ranking_roic"]
dados_empresas["ranking_final"] = dados_empresas.groupby("data")["ranking_final"].rank()

dados_empresas

# Passo 6: Criar as carteiras. 

In [ ]:
dados_empresas = dados_empresas[dados_empresas["ranking_final"] <= 10]

dados_empresas

# Passo 7: Calcular a rentabilidade por carteira.

In [ ]:
rentabilidades_carteiras = dados_empresas.groupby("data")["retorno_mensal"].mean()
rentabilidades_carteiras = rentabilidades_carteiras.to_frame()

rentabilidades_carteiras

# Passo 8: Calcular a rentabilidade do modelo.

In [ ]:
rentabilidades_carteiras["magic_formula"] = (rentabilidades_carteiras["retorno_mensal"] + 1).cumprod() - 1

rentabilidades_carteiras = rentabilidades_carteiras.shift(1)

rentabilidades_carteiras = rentabilidades_carteiras.dropna()

rentabilidades_carteiras

# Passo 9: Calcular a rentabilidade do ibovespa no mesmo período.

In [ ]:
ibov = pd.read_csv("ibov.csv")

ibov

In [ ]:
retornos_ibov = ibov["fechamento"].pct_change().dropna()
retorno_acum_ibov = (1 + retornos_ibov).cumprod() - 1
rentabilidades_carteiras["ibovespa"] = retorno_acum_ibov.values
rentabilidades_carteiras = rentabilidades_carteiras.drop("retorno_mensal", axis = 1)

rentabilidades_carteiras

# Passo 10: Observar resultados de retorno.

In [ ]:
qs.extend_pandas()

rentabilidades_carteiras.index = pd.to_datetime(rentabilidades_carteiras.index)

rentabilidades_carteiras

In [ ]:
rentabilidades_carteiras["magic_formula"].plot_monthly_heatmap()
rentabilidades_carteiras["ibovespa"].plot_monthly_heatmap()

In [ ]:
rentabilidades_carteiras.plot(figsize = (7, 4))

# Passo 11 - Otimizar nosso modelo com outros indicadores. 

E se nós adicionássemos um indicador de momentum no modelo?

# Passo 12 - Observar métricas de risco, janelas móveis de retorno, eventos de estresse...